In [ ]:
!pip install lenskit

import lenskit.datasets as ds
import pandas as pd

!git clone https://github.com/devkcodes/course_recomendation_dataset.git

data = ds.MovieLens('course_recomendation_dataset/')

print("dataset loaded")

fatal: destination path 'course_recomendation_dataset' already exists and is not an empty directory.
dataset loaded


In [ ]:
print('table-check')
rows_to_show = 5  

data.ratings.head(rows_to_show)

table-check


,user,item,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
joined_data = data.ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data.head(rows_to_show)


In [ ]:
minimum_to_include = 20 #<-- You can try changing this minimum to include movies rated by fewer or more people

average_ratings = (data.ratings).groupby(['item']).mean()
rating_counts = (data.ratings).groupby(['item']).count()
average_ratings = average_ratings.loc[rating_counts['rating'] > minimum_to_include]
sorted_avg_ratings = average_ratings.sort_values(by="rating", ascending=False)
joined_data = sorted_avg_ratings.join(data.movies['genres'], on='item')
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[3:]]

not_enough_data=joined_data

print("RECOMMENDED FOR ANYBODY:")
joined_data.head(rows_to_show)

RECOMMENDED FOR ANYBODY:


,genres,title
item,,
318,Crime|Drama,"Shawshank Redemption, The (1994)"
922,Drama|Film-Noir|Romance,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
898,Comedy|Drama|Romance,"Philadelphia Story, The (1940)"
475,Drama,In the Name of the Father (1993)
1204,Adventure|Drama|War,Lawrence of Arabia (1962)


In [ ]:
import csv

devansh_rating_dict = {}
dhruv_rating_dict = {}
check3_rating_dict={}

with open("/content/course_recomendation_dataset/devansh-movie-ratings.csv", newline='') as csvfile:
  ratings_reader = csv.DictReader(csvfile)
  for row in ratings_reader:
    if ((row['ratings'] != "") and (float(row['ratings']) > 0) and (float(row['ratings']) < 6)):
      devansh_rating_dict.update({int(row['item']): float(row['ratings'])})
      
with open("/content/course_recomendation_dataset/dhruv-movie-ratings.csv", newline='') as csvfile:
  ratings_reader = csv.DictReader(csvfile)
  for row in ratings_reader:
    if ((row['ratings'] != "") and (float(row['ratings']) > 0) and (float(row['ratings']) < 6)):
      dhruv_rating_dict.update({int(row['item']): float(row['ratings'])})

      
with open("/content/course_recomendation_dataset/check3-movie-ratings.csv", newline='') as csvfile:
  ratings_reader = csv.DictReader(csvfile)
  for row in ratings_reader:
    if ((row['ratings'] != "") and (float(row['ratings']) > 0) and (float(row['ratings']) < 6)):
      check3_rating_dict.update({int(row['item']): float(row['ratings'])})
     
print("Rating dictionaries assembled!")
print("Sanity check:")
print("\tdevansh's rating for 1197 (The Princess Bride) is " + str(devansh_rating_dict[1197]))
print("\tdhruv's rating for 1197 (The Princess Bride) is " + str(dhruv_rating_dict[1197]))
print("\tcheck3's rating for 1197 (The Princess Bride) is " + str(check3_rating_dict[141]))

print(ratings_reader)

Rating dictionaries assembled!
Sanity check:
	devansh's rating for 1197 (The Princess Bride) is 4.5
	dhruv's rating for 1197 (The Princess Bride) is 3.5
	check3's rating for 1197 (The Princess Bride) is 5.0


In [ ]:
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

num_recs = 10

user_user = UserUser(15, min_nbrs=3)
algo = Recommender.adapt(user_user)
algo.fit(data.ratings)

print("Set up User-User")

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB
BLAS using multiple threads - can cause oversubscription
found 2 potential runtime problems - see https://boi.st/lkpy-perf


Set up User-User


In [ ]:
devansh_recs = algo.recommend(-1, num_recs, ratings=pd.Series(devansh_rating_dict))  


joined_data = devansh_recs.join(data.movies['genres'], on='item')      
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[2:]]
print("\n\nRECOMMENDATIONS FOR DEVANSH:")
joined_data



RECOMMENDATIONS FOR DEVANSH:


,genres,title
0,Comedy|Drama,"Last Detail, The (1973)"
1,Comedy,Love and Death (1975)
2,Drama,Before Night Falls (2000)
3,Drama,"Magdalene Sisters, The (2002)"
4,Drama|Horror|Mystery|Sci-Fi|Thriller,Black Mirror: White Christmas (2014)
5,Action|Animation|Drama|Fantasy|Sci-Fi,Neon Genesis Evangelion: The End of Evangelion...
6,Action|Adventure|Thriller,Raiders of the Lost Ark: The Adaptation (1989)
7,Comedy|Drama|Romance,Submarine (2010)
8,Adventure|Drama,Nebraska (2013)
9,Documentary,"Endless Summer, The (1966)"


In [ ]:
joined_data
check3_recs = algo.recommend(-1, num_recs, ratings=pd.Series(check3_rating_dict))  
joined_data = check3_recs.join(data.movies['genres'], on='item')      
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[2:]]
joined_data.head(10)

if len(check3_rating_dict)<10:
  print("\n\nRECOMMENDATIONS FOR check3:")
  not_enough_data.head(rows_to_show)
else:
  print("\n\nRECOMMENDATIONS FOR check3:")
  joined_data.head(rows_to_show)



RECOMMENDATIONS FOR check3:


In [ ]:
dhruv_recs = algo.recommend(-1, num_recs, ratings=pd.Series(dhruv_rating_dict))  #Here, -1 tells it that it's not an existing user in the set, that we're giving new ratings, while 10 is how many recommendations it should generate

joined_data = dhruv_recs.join(data.movies['genres'], on='item')      
joined_data = joined_data.join(data.movies['title'], on='item')
joined_data = joined_data[joined_data.columns[2:]]
print("RECOMMENDATIONS FOR DHRUV:")
joined_data

RECOMMENDATIONS FOR DHRUV:


,genres,title
0,Comedy,The Night Before (2015)
1,Adventure|Drama|Sci-Fi,"Day of the Doctor, The (2013)"
2,Drama|Fantasy|Romance,Wristcutters: A Love Story (2006)
3,Comedy|Musical,Holiday Inn (1942)
4,Comedy,Outside Providence (1999)
5,Comedy|Romance,Adam's Rib (1949)
6,Drama,Reign Over Me (2007)
7,Drama,Guess Who's Coming to Dinner (1967)
8,Drama,Half Nelson (2006)
9,Comedy,Fired Up (2009)
